In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

url = 'https://quality.data.gov.tw/dq_download_csv.php?nid=16481&md5_url=2f75965a69d71fbc5a15a128b36d84c8'
df = pd.read_csv(url, encoding='utf-8')

# 色彩設定
domain_colors = {'火力':'steelblue', '核能':'crimson', '再生能源':'forestgreen'}
sub_colors = {
    '火力_燃煤_全國':'saddlebrown', '火力_燃油_全國':'darkorange', '火力_燃氣_全國':'gold',
    '再生能源_慣常水力_全國':'deepskyblue','再生能源_地熱_全國':'darkred',
    '再生能源_太陽光電_全國':'orange','再生能源_風力_全國':'limegreen',
    '再生能源_生質能_全國':'olive','再生能源_廢棄物_全國':'gray'
}

years = df['日期(年)'].tolist()

def compute_props(year):
    row = df[df['日期(年)'] == year].iloc[0]
    props = {
        '火力': row['火力_全國'],
        '核能': row['核能_全國'],
        '再生能源': row['再生能源_全國']
    }
    total = sum(props.values())
    return {k: v / total * 100 for k, v in props.items()}

def format_label(k):
    return k.replace('再生能源_', '').replace('火力_', '').replace('_全國','')

# widgets
year_dd = widgets.Dropdown(options=years, description='年份:')
energy_dd = widgets.Dropdown(options=['火力', '再生能源'], description='主能源:')

main_fig_out = widgets.Output()
sub_fig_out = widgets.Output()

def update_main_fig(year):
    props = compute_props(year)
    labels = list(props.keys())
    values = list(props.values())
    colors = [domain_colors[k] for k in labels]

    fig = go.Figure(go.Pie(
        labels=labels, values=values,
        marker=dict(colors=colors),
        textinfo='label+percent',
        textposition='inside'
    ))
    fig.update_layout(title_text=f"{year} 能源發電佔比")
    main_fig_out.clear_output()
    with main_fig_out:
        display(fig) 

def update_sub_fig(year, energy):
    if energy == '火力':
        keys = ['火力_燃煤_全國', '火力_燃油_全國', '火力_燃氣_全國']
    elif energy == '再生能源':
        keys = ['再生能源_慣常水力_全國', '再生能源_地熱_全國',
                '再生能源_太陽光電_全國', '再生能源_風力_全國',
                '再生能源_生質能_全國', '再生能源_廢棄物_全國']
    else:
        return

    row = df[df['日期(年)'] == year].iloc[0]
    vals = [row[k] for k in keys]
    labels = [format_label(k) for k in keys]
    colors = [sub_colors[k] for k in keys]

    fig = go.Figure(go.Pie(
        labels=labels, values=vals,
        marker=dict(colors=colors),
        textinfo='label+percent',
        textposition='inside'
    ))
    fig.update_layout(title_text=f"{year} {energy} 細項佔比")
    sub_fig_out.clear_output()
    with sub_fig_out:
        display(fig)

def on_change(change=None):
    if year_dd.value and energy_dd.value:
        update_main_fig(year_dd.value)
        update_sub_fig(year_dd.value, energy_dd.value)

year_dd.observe(on_change, names='value')
energy_dd.observe(on_change, names='value')

btn = widgets.Button(description='顯示圖表')

def on_btn_click(b):
    on_change()

btn.on_click(on_btn_click)

widgets.VBox([year_dd, energy_dd, btn, main_fig_out, sub_fig_out])